# 11.3.1 Helmholtz solver using Brakhage-Werner formulation

The solution is represented using

$$
u = (K + i \kappa V) \phi,
$$

where $\phi$ solve the boundary integral equation
$$
    \big( \tfrac{1}{2} + K + i \kappa V \big) g = u_{in} \qquad \text{on} \, \Gamma
$$


In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *

In [ ]:
order=4

In [ ]:
screen = WorkPlane(Axes( (0,0,0), Z, X)).RectangleC(15,15).Face()

sp = Fuse(Sphere( (0,0,0), pi).faces)
screen.faces.name="screen"
sp.faces.name="sphere"
shape = Compound([screen,sp])

mesh = shape.GenerateMesh(maxh=1).Curve(order)
Draw (mesh);

In [ ]:
fes_sphere = Compress(SurfaceL2(mesh, order=order, complex=True, definedon=mesh.Boundaries("sphere")))
fes_screen = Compress(SurfaceL2(mesh, order=order, dual_mapping=True, complex=True, definedon=mesh.Boundaries("screen")))
print ("ndof_sphere = ", fes_sphere.ndof, "ndof_screen =", fes_screen.ndof)

In [ ]:
kappa = 5

In [ ]:
with TaskManager():
    V = HelmholtzSingleLayerPotentialOperator(fes_sphere, fes_sphere, kappa=kappa, intorder=10)
    K = HelmholtzDoubleLayerPotentialOperator(fes_sphere, fes_sphere, kappa=kappa, intorder=10)
    C = HelmholtzCombinedFieldOperator(fes_sphere, fes_sphere, kappa=kappa, intorder=10)
    u,v  = fes_sphere.TnT()
    Id = BilinearForm(u*v*ds).Assemble()

In [ ]:
lhs = 0.5 * Id.mat + C.mat
source = exp(1j * kappa * x) 
rhs = LinearForm(-source*v*ds).Assemble()

In [ ]:
gfu = GridFunction(fes_sphere)
pre = BilinearForm(u*v*ds, diagonal=True).Assemble().mat.Inverse()
with TaskManager():
    gfu.vec[:] = solvers.GMRes(A=lhs, b=rhs.vec, pre=pre, maxsteps=10)

In [ ]:
Draw (gfu, order=5, min=-1, max=1);

## prostprocessing on screen

In [ ]:
uscat = GridFunction(fes_screen)
with TaskManager():
    uscat.Set(1j*kappa*V.GetPotential(gfu)-K.GetPotential(gfu) , definedon=mesh.Boundaries("screen"))

In [ ]:
Draw (uscat, mesh, min=-1,max=1, animate_complex=True, order=4);

In [ ]:
uin = mesh.BoundaryCF( {"screen": source }, default=0)
print ("Total field")
Draw (uin-uscat, mesh, min=-1,max=1, animate_complex=True, order=4);

In [ ]:
print ("Scattered field only:")
Draw (uscat, mesh, min=-1,max=1, animate_complex=True, order=6);